In [ ]:
from hydromt_sfincs import SfincsModel, utils
import numpy as np
from os.path import join
import hydromt
import xarray as xr

## bias correct flood depths

In [ ]:
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"

In [ ]:
ds = xr.open_zarr(join(rdir, 'hmax.zarr')).swap_dims({'index':'scen'})
da = ds['hmax'].fillna(0)
da.attrs.update(_FillValue=da.attrs['nodatavals'][0])
da

In [ ]:
mod0 = SfincsModel(join(mdir, '00_base_riv'), mode='r')
rivmsk = mod0.staticmaps['rivmsk'].raster.flipud()==1

In [ ]:
# bias correct for Q and H but make sure to keep P
nodata = da.attrs['_FillValue']
# get Q2 + MSHW map for bias correction
bias, da0 = 'q2', da.sel(scen=['qb002_qp002_h000_p000']).squeeze(drop=True)
mask = np.logical_and(da0!=nodata, ~rivmsk)
# account for minimal P in all scenarios
pscen = [f'qb000_qp000_h000_p{s[-3:]}' for s in da.scen.values]
dap = da.sel(scen=pscen).reset_coords(drop=True).drop('scen')
dap = np.maximum(0, dap - da.sel(scen=['qb000_qp000_h000_p000']).squeeze(drop=True))
# take maximum of pscen and bias corrected scens
da1 = np.maximum(dap, da-da0).where(mask, 0)
# da1.attrs.update(**da.attrs)
# da1 = da1.chunk({'x':-1, 'y':-1, 'scen':10})

In [ ]:
# ds_out = xr.merge([ds.drop_vars('hmax'), da1]).swap_dims({'scen': 'index'})
ds_out = da1.to_dataset().swap_dims({'scen': 'index'})
ds_out.attrs = {}
ds_out.chunk({'index':50, 'x':250, 'y':250}).drop(['spatial_ref', 'band']).to_zarr(join(rdir, 'hmax_bias_corrected.zarr'))

In [ ]:
# save bias corrected floodmaps
for rp in  np.array([0,2,5,10,50,100,500], dtype=int):
    scens = [
        f'qb{rp:03d}_qp000_h000_p000', 
        f'qb000_qp{rp:03d}_h000_p000', 
        f'qb000_qp000_h{rp:03d}_p000', 
        f'qb000_qp000_h000_p{rp:03d}', 
        f'qb{rp:03d}_qp{rp:03d}_h{rp:03d}_p{rp:03d}'
    ]
    for scen in scens:
        da_scen = da1.sel(scen=scen)
        fn_out = join(rdir, 'hmax', f'{scen}.tif')
        da_scen.raster.flipud().raster.to_raster(fn_out, compress='lzw')